In [1]:
from Dataload import dataload

from module import transformer,video_model,invres

import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils
#Batchfy

from constant import EMOTIPATH
#Counts number of correct label
def num_correct(prediction,labels):
    correct=0
    for i,(pred_label,label) in enumerate(zip(prediction,labels)):
        if (pred_label.item()==label.item()):
            correct +=1
    return correct

import os

Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video=os.path.join(EMOTIPATH,"Train")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video=os.path.join(EMOTIPATH,"Val")
train_data=dataload.Video_Frame_Data(Train_label,base_path_v=Train_video,strict_num=26)
valid_data=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=26)



            
train_dataloader = DataLoader(train_data, batch_size=64
                       , num_workers=0,shuffle=False)

valid_dataloader = DataLoader(valid_data, batch_size=64
                       , num_workers=0,shuffle=False)

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [3]:
model=video_model.Video_Feature(joint=False,grayscale=False,classfication=True,frame=24,skip_process=False)

model=model.to(device)

for name, child in model.named_children():
   if not name in ['densenet']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False
model.eval()           
print("ready")

densenet is frozen
fc2 is unfrozen
fc3 is unfrozen
posencoding is unfrozen
transformer is unfrozen
ready


In [4]:
len(train_data)

2661

In [5]:
def preprocess(): 
    data="Embedding\n"
    import numpy as np        
    f=open('train.csv','ab')
    f.write(data.encode())
    model.eval()
    for i_batch, (sample_batched,_,label) in enumerate(train_dataloader):
        batch_size=sample_batched.size(0)
        sample_batched=sample_batched.to(device)
        sample_output=model(sample_batched)
        np.savetxt(f,(sample_output.reshape(batch_size,-1).float().cpu().numpy()),fmt="%f")

    f.close()

In [6]:
preprocess()

In [ ]:
import pandas as pd
import numpy as np
class Video_Embedding_Data(Dataset):
    def __init__(self, embed_file,label_file,frame=24):
        self._table_embedding = pd.read_csv(embed_file)
        self._table_label = pd.read_csv(label_file,delimiter=' ')
        self.frame=frame
    def __len__(self):
        return len(self._table_embedding)

    def __getitem__(self, idx):
        embedding=torch.from_numpy(np.array(debug._table_embedding.Embedding[idx].split(),dtype=float)).reshape((self.frame,-1))
        labels = torch.from_numpy(np.array(self._table_label.Label[idx]))-1
        
        return (embedding,labels)
    

In [ ]:
debug=Video_Embedding_Data("valid.csv",Val_labels)

In [ ]:
real_output=model(train_data[0][0].unsqueeze(0).to(device))

In [ ]:
debug[0]

In [ ]:
real_output[0]

In [ ]:
len(debug)

In [7]:
def preprocess(): 
    data="Embedding\n"
    import numpy as np        
    f=open('valid.csv','ab')
    f.write(data.encode())
    model.eval()
    for i_batch, (sample_batched,_,label) in enumerate(valid_dataloader):
        batch_size=sample_batched.size(0)
        sample_batched=sample_batched.to(device)
        sample_output=model(sample_batched)
        np.savetxt(f,(sample_output.reshape(batch_size,-1).float().cpu().numpy()),fmt="%f")

    f.close()

In [8]:


preprocess()